In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving global_cancer_patients_2015_2024.csv to global_cancer_patients_2015_2024 (1).csv


In [ ]:
df =pd.read_csv('global_cancer_patients_2015_2024.csv', header=None, sep=',')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Patient_ID,Age,Gender,Country_Region,Year,Genetic_Risk,Air_Pollution,Alcohol_Use,Smoking,Obesity_Level,Cancer_Type,Cancer_Stage,Treatment_Cost_USD,Survival_Years,Target_Severity_Score
1,PT0000000,71,Male,UK,2021,6.4,2.8,9.5,0.9,8.7,Lung,Stage III,62913.44,5.9,4.92
2,PT0000001,34,Male,China,2021,1.3,4.5,3.7,3.9,6.3,Leukemia,Stage 0,12573.41,4.7,4.65
3,PT0000002,80,Male,Pakistan,2023,7.4,7.9,2.4,4.7,0.1,Breast,Stage II,6984.33,7.1,5.84
4,PT0000003,40,Male,UK,2015,1.7,2.9,4.8,3.5,2.7,Colon,Stage I,67446.25,1.6,3.12


- check columns ( names, numbers , data types )

In [ ]:
df.shape

(50001, 15)

- check for nan values
- drop any if found and test again to check

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       50001 non-null  object
 1   1       50001 non-null  object
 2   2       50001 non-null  object
 3   3       50001 non-null  object
 4   4       50001 non-null  object
 5   5       50001 non-null  object
 6   6       50001 non-null  object
 7   7       50001 non-null  object
 8   8       50001 non-null  object
 9   9       50001 non-null  object
 10  10      50001 non-null  object
 11  11      50001 non-null  object
 12  12      50001 non-null  object
 13  13      50001 non-null  object
 14  14      50001 non-null  object
dtypes: object(15)
memory usage: 5.7+ MB


In [ ]:
df.sum

<bound method DataFrame.sum of                0    1       2               3     4             5   \
0      Patient_ID  Age  Gender  Country_Region  Year  Genetic_Risk   
1       PT0000000   71    Male              UK  2021           6.4   
2       PT0000001   34    Male           China  2021           1.3   
3       PT0000002   80    Male        Pakistan  2023           7.4   
4       PT0000003   40    Male              UK  2015           1.7   
...           ...  ...     ...             ...   ...           ...   
49996   PT0049995   80    Male        Pakistan  2023           2.3   
49997   PT0049996   40  Female             USA  2018           6.4   
49998   PT0049997   74    Male        Pakistan  2015           6.2   
49999   PT0049998   21    Male          Brazil  2018           4.0   
50000   PT0049999   22    Male        Pakistan  2023           5.1   

                  6            7        8              9            10  \
0      Air_Pollution  Alcohol_Use  Smoking  Obesity_Level  Cancer_Type   
1                2.8          9.5      0.9            8.7         Lung   
2                4.5          3.7      3.9            6.3     Leukemia   
3                7.9          2.4      4.7            0.1       Breast   
4                2.9          4.8      3.5            2.7        Colon   
...              ...          ...      ...            ...          ...   
49996            7.5          2.8      3.8            2.9     Leukemia   
49997            3.5          2.9      9.0            9.8        Colon   
49998            1.6          8.7      4.7            4.0       Breast   
49999            6.5          7.6      8.6            8.1        Liver   
50000            9.8          3.2      0.0            0.7        Liver   

                 11                  12              13                     14  
0      Cancer_Stage  Treatment_Cost_USD  Survival_Years  Target_Severity_Score  
1         Stage III            62913.44             5.9                   4.92  
2           Stage 0            12573.41             4.7                   4.65  
3          Stage II             6984.33             7.1                   5.84  
4           Stage I            67446.25             1.6                   3.12  
...             ...                 ...             ...                    ...  
49996       Stage 0             71463.7             1.0                   3.63  
49997       Stage I            49619.66             0.4                   6.03  
49998       Stage 0             5338.25             2.7                   6.02  
49999      Stage IV            45100.47             7.5                   6.54  
50000       Stage 0            64023.82             7.4                   3.76  

[50001 rows x 15 columns]>

- describe the data
- Standardize units (e.g., convert sleep duration to consistent units)

- Summary statistics (mean, median, distributions).

- Initial visualizations (histograms, box plots for key variables)